In [1]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [2]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

In [3]:
#Build the CNN Model 
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Flatten,Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Dropout

In [4]:
train_gen = ImageDataGenerator(rescale=(1./255),horizontal_flip=True,shear_range=0.2) 
test_gen = ImageDataGenerator(rescale=(1./255))

In [5]:
#passing a value using another variables
train = train_gen.flow_from_directory('Dataset(original)/Train',
                                      target_size=(224,224),
                                      class_mode='categorical',
                                      batch_size=8)

test = test_gen.flow_from_directory('Dataset(original)/test',
                                    target_size=(224,224),
                                    class_mode='categorical',
                                    batch_size=8)

Found 655 images belonging to 17 classes.
Found 183 images belonging to 17 classes.


In [6]:
# Load the pre-trained DenseNet121 model without the top/fully-connected layers
densenet = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [34]:
'''# Set DenseNet121 layers as non-trainable
for layer in densenet.layers:
    layer.trainable = False

x = densenet.output
x = GlobalAveragePooling2D()(x)

# Build the combined model
cm_DN_CNN = Dense(40, activation='relu')(x)
cm_DN_CNN = Dense(80, activation='relu')(x)
cm_DN_CNN = Dense(160, activation='relu')(x)
cm_DN_CNN = Dense(80, activation='relu')(x)
cm_DN_CNN = Dense(18, activation='softmax')(x)'''

In [7]:
# Set DenseNet121 layers as non-trainable
for layer in densenet.layers:
    layer.trainable = False

x = densenet.output

x = Convolution2D(50, (3, 3), activation='relu', padding='same')(x)
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)

# Build the combined model
output = Dense(80, activation='relu')(x)
output = Dense(160, activation='relu')(output)
output = Dense(320, activation='relu')(output)
output = Dense(160, activation='relu')(output)
output = Dense(17, activation='softmax')(output)

In [8]:
model = Model(inputs=densenet.input,outputs = output)

In [9]:
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [10]:
model.fit(train,validation_data=test,epochs=40,
                    steps_per_epoch=len(train),
                    validation_steps = len(test))

Epoch 1/40
82/82 [==============================] - 44s 454ms/step - loss: 2.8406 - accuracy: 0.0855 - val_loss: 2.7757 - val_accuracy: 0.1803
Epoch 2/40
82/82 [==============================] - 37s 448ms/step - loss: 2.6582 - accuracy: 0.1359 - val_loss: 2.4743 - val_accuracy: 0.2186
Epoch 3/40
82/82 [==============================] - 37s 453ms/step - loss: 2.4968 - accuracy: 0.1725 - val_loss: 2.3884 - val_accuracy: 0.2623
Epoch 4/40
82/82 [==============================] - 38s 459ms/step - loss: 2.3185 - accuracy: 0.2458 - val_loss: 2.1782 - val_accuracy: 0.2732
Epoch 5/40
82/82 [==============================] - 40s 485ms/step - loss: 2.1928 - accuracy: 0.2809 - val_loss: 2.1664 - val_accuracy: 0.2568
Epoch 6/40
82/82 [==============================] - 39s 474ms/step - loss: 2.0523 - accuracy: 0.3160 - val_loss: 1.8515 - val_accuracy: 0.3716
Epoch 7/40
82/82 [==============================] - 37s 456ms/step - loss: 1.9419 - accuracy: 0.3435 - val_loss: 1.9405 - val_accuracy: 0.3552

In [12]:
# Save the trained model
model.save('DenseNet121.h5')

In [13]:
train.class_indices

{'Darier_s disease': 0,
 'Muehrck-e_s lines': 1,
 'aloperia areata': 2,
 'beau_s lines': 3,
 'bluish nail': 4,
 'clubbing': 5,
 'eczema': 6,
 'half and half nailes (Lindsay_s nails)': 7,
 'koilonychia': 8,
 'leukonychia': 9,
 'onycholycis': 10,
 'pale nail': 11,
 'red lunula': 12,
 'splinter hemmorrage': 13,
 'terry_s nail': 14,
 'white nail': 15,
 'yellow nails': 16}

In [14]:
#Testing
import numpy as np
from tensorflow.keras.preprocessing import image

In [15]:
img=image.load_img('Dataset(original)/train/bluish nail/1.PNG',target_size=(224,224))

In [16]:
img = image.img_to_array(img)
img = np.expand_dims(img,axis=0)
model.predict(img)
pred = np.argmax(model.predict(img))
output = ['Darier_s disease','Muehrck-e_s lines','Onychogryphosis','Onycholycis_NailPsoriasis',
          'aloperia areata','beau_s lines','bluish nail', 'clubbing','eczema','half and half nailes (Lindsay_s nails)',
          'koilonychia','leukonychia','pale nail','red lunula',
          'splinter hemmorrage_Acral Lentiginous Melanoma',
          'terry_s nail_WhiteNails','yellow nails']
print(output[pred])

1/1 [==============================] - 0s 75ms/step
terry_s nail_WhiteNails
